In [ ]:
!pip install -U segmentation-models

  Cloning https://github.com/divamgupta/image-segmentation-keras.git to /tmp/pip-req-build-xyzn3sht
  Running command git clone -q https://github.com/divamgupta/image-segmentation-keras.git /tmp/pip-req-build-xyzn3sht
     |████████████████████████████████| 3.3MB 4.6MB/s 
  Created wheel for keras-segmentation: filename=keras_segmentation-0.3.0-cp36-none-any.whl size=41436 sha256=c32a9ece7fbab33905a29e97d2bf55348acd315ad2d473120b1ed51d7f82c077
  Stored in directory: /tmp/pip-ephem-wheel-cache-n261bhae/wheels/0a/4e/8b/d3f9088958a351a72f48ede4558836b78a03c0c124f81f41ed
Successfully built keras-segmentation
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1


In [ ]:
import tensorflow as tf
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator  
from keras.applications import densenet  
from keras.models import Sequential, Model, load_model  
from keras.optimizers import Adam, SGD
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Input, AveragePooling2D, InputLayer
from keras.layers import Activation, Dropout, Flatten, Dense  
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback  
from keras import regularizers  
from keras import backend as K 
from PIL import Image
import numpy as np
from keras.models import model_from_json

import segmentation_models as sm

import matplotlib.pyplot as plt
tf.__version__

Segmentation Models: using `keras` framework.


'1.15.0'

In [ ]:
def rmse(y_pred, y_true):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [ ]:
def pred_rmse(y_pred, y_true):
  return np.sqrt(np.mean((y_pred.values-y_true.squeeze())**2))

In [ ]:
percentages = pd.read_csv('/content/drive/My Drive/Master/DeepLearning/UNIMIB2016/percentage.csv')

In [ ]:
test_index = int(len(percentages)*0.75)
train = percentages.iloc[:test_index,:]
test = percentages.iloc[test_index:,:]

In [ ]:
validation_split = 0.1
train_data_dir = '/content/drive/My Drive/Master/DeepLearning/UNIMIB2016/images'
img_height, img_width = 640//2, 256 #It's necessary make height and width divisible by 32
batch_size = 32
nb_epochs = 100

train_datagen = ImageDataGenerator(rescale=1./255,
    validation_split=validation_split, horizontal_flip=True,
     vertical_flip=True, rotation_range=90, brightness_range=[0.2,1.0])

test_datagen = ImageDataGenerator(rescale=1./255)  

train_generator = train_datagen.flow_from_dataframe(
     dataframe=train,
    directory=train_data_dir,
    x_col="image_name",
    y_col="food_pixels",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="raw",
    #color_mode = "grayscale",
    subset='training')

validation_generator = train_datagen.flow_from_dataframe(
    dataframe=train,
    directory=train_data_dir,
    x_col="image_name",
    y_col="food_pixels",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="raw",
    #color_mode = "grayscale",
    subset='validation')

test_generator = test_datagen.flow_from_dataframe(
     dataframe=test,
    directory=train_data_dir,
    x_col="image_name",
    y_col="food_pixels",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="raw",shuffle=False)

Found 693 validated image filenames.
Found 77 validated image filenames.
Found 257 validated image filenames.


In [ ]:
base_model = sm.Unet('resnet34', input_shape=(img_height, img_width, 3), encoder_weights='imagenet')

In [ ]:
x = base_model.get_layer('final_conv').output

x = Flatten()(x)
x = Dense(32)(x)
x = Activation("relu")(x)
x = BatchNormalization(axis=-1)(x)
x = Dropout(0.3)(x)

x = Dense(16)(x)
x = Activation("relu")(x)

predictions = Dense(1, init='glorot_uniform', activation="linear")(x)

model = Model(input=base_model.input, output=predictions)

opt = Adam(lr=1e-3)

es = EarlyStopping(monitor='val_loss', patience=10, mode='auto', restore_best_weights=True)
r_lr = ReduceLROnPlateau(monitor='val_loss', patience=5, mode='auto',  min_lr=1e-6)
model.compile(loss="mean_squared_error", optimizer=opt, metrics=['mae', 'mse', rmse])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="linear", kernel_initializer="glorot_uniform")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("da..., outputs=Tensor("de...)`
  del sys.path[0]


In [ ]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 320, 256, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 320, 256, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_103 (ZeroPadding (None, 326, 262, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 160, 128, 64) 9408        zero_padding2d_103[0][0]         
____________________________________________________________________________________________

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs, callbacks = [es, r_lr])

Epoch 1/100
21/21 [==============================] - 388s 18s/step - loss: 200.3044 - mean_absolute_error: 13.1875 - mean_squared_error: 200.3044 - rmse: 14.1141 - val_loss: 156.5642 - val_mean_absolute_error: 11.0602 - val_mean_squared_error: 156.5642 - val_rmse: 12.5125
Epoch 2/100
21/21 [==============================] - 36s 2s/step - loss: 147.1656 - mean_absolute_error: 11.0856 - mean_squared_error: 147.1656 - rmse: 12.0894 - val_loss: 115.3661 - val_mean_absolute_error: 9.2075 - val_mean_squared_error: 115.3661 - val_rmse: 10.6984
Epoch 3/100
21/21 [==============================] - 20s 940ms/step - loss: 115.9657 - mean_absolute_error: 9.5352 - mean_squared_error: 115.9657 - rmse: 10.7173 - val_loss: 66.1097 - val_mean_absolute_error: 6.6266 - val_mean_squared_error: 66.1097 - val_rmse: 8.1302
Epoch 4/100
21/21 [==============================] - 21s 1s/step - loss: 74.7568 - mean_absolute_error: 7.4374 - mean_squared_error: 74.7568 - rmse: 8.6121 - val_loss: 37.1682 - val_mean_a

In [ ]:
import numpy as np
print("RMSE: "+str(pred_rmse(test["food_pixels"], model.predict(test_generator))))

RMSE: 1.8699741775743168


In [ ]:
model.save('/content/drive/My Drive/Master/DeepLearning/UNIMIB2016/models/unet_transfer_finetuning_model.h5')